In [1]:
import subprocess

import pandas as pd

from helpers import cleanup_dollar_value

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
# https://www.chop.edu/centers-programs/billing-and-insurance/understanding-hospital-charges
ccn = "393303"
url = "https://media.chop.edu/data/files/finance/23-1352166_ChildrensHospitalofPhiladelphia_standardcharges.csv"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein

'23-1352166'

In [4]:
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘23-1352166_ChildrensHospitalofPhiladelphia_standardcharges.csv’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://media.chop.edu/data/files/finance/23-1352166_ChildrensHospitalofPhiladelphia_standardcharges.csv', '-O', '23-1352166_ChildrensHospitalofPhiladelphia_standardcharges.csv'], returncode=1)

In [5]:
df_in = pd.read_csv(filename, skiprows=range(1,12), encoding='latin-1')
df_in.columns = df_in.iloc[0].to_list()
df_in = df_in.iloc[1:]
df_in

/tmp/ipykernel_13877/356144194.py:1: DtypeWarning: Columns (3,4,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_in = pd.read_csv(filename, skiprows=range(1,12), encoding='latin-1')


,Inpatient/Outpatient/Drug,Location (All Locations Excluding Lancaster),Code Type,Lab Code,Procedure,Code,Procedure Description,NDC,Gross Charge,De-Identified Minimum,De-Identified Maximum,Third Party Payer,Payer-Specific Negotiated Charge
1,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,"80,887.44",NaN,NaN,NaN,NaN
2,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,"70,558.50",NaN,NaN,NaN,NaN
3,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,"127,150.82",NaN,NaN,NaN,NaN
4,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,"438,620.29",NaN,NaN,NaN,NaN
5,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,"277,411.38",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,NaN,NaN,NaN,Preferred Health Care,"278,286.91"
269965,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"9,429.04"
269966,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"31,395.05"
269967,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"151,876.07"


In [6]:
df_mid = pd.DataFrame(df_in)
df_mid = df_mid.rename(columns={
    'Inpatient/Outpatient/Drug': 'setting',
    'Location (All Locations Excluding Lancaster)': 'location',
    'Code Type': 'line_type',
    'Procedure': 'local_code',
    'Code': 'code',
    'Procedure Description': 'description',
    'NDC': 'ndc',
    ' Payer-Specific Negotiated Charge ': 'standard_charge',
    'Third Party Payer': 'payer_name',
    'Lab Code': 'additional_generic_notes'
})

df_mid

,setting,location,line_type,additional_generic_notes,local_code,code,description,ndc,Gross Charge,De-Identified Minimum,De-Identified Maximum,payer_name,standard_charge
1,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,"80,887.44",NaN,NaN,NaN,NaN
2,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,"70,558.50",NaN,NaN,NaN,NaN
3,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,"127,150.82",NaN,NaN,NaN,NaN
4,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,"438,620.29",NaN,NaN,NaN,NaN
5,Inpatient,Children's Hospital of Philadelphia,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,"277,411.38",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,NaN,NaN,NaN,Preferred Health Care,"278,286.91"
269965,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"9,429.04"
269966,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"31,395.05"
269967,Inpatient,Children's Seashore House of CHOP,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"151,876.07"


In [7]:
set(df_mid['location'].to_list())

{"Children's Hospital of Philadelphia", "Children's Seashore House of CHOP"}

In [8]:
df_mid['hospital_id'] = ccn
df_mid.loc[df_mid['location'] == "Children's Seashore House of CHOP", 'hospital_id'] = '393300'
del df_mid['location']
df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,Gross Charge,De-Identified Minimum,De-Identified Maximum,payer_name,standard_charge,hospital_id
1,Inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,"80,887.44",NaN,NaN,NaN,NaN,393303
2,Inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,"70,558.50",NaN,NaN,NaN,NaN,393303
3,Inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,"127,150.82",NaN,NaN,NaN,NaN,393303
4,Inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,"438,620.29",NaN,NaN,NaN,NaN,393303
5,Inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,"277,411.38",NaN,NaN,NaN,NaN,393303
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,Inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,NaN,NaN,NaN,Preferred Health Care,"278,286.91",393300
269965,Inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"9,429.04",393300
269966,Inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"31,395.05",393300
269967,Inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"151,876.07",393300


In [9]:
df_mid.loc[df_mid['setting'] == 'Inpatient', 'setting'] = 'inpatient'
df_mid.loc[df_mid['setting'] == 'Outpatient', 'setting'] = 'outpatient'
df_mid.loc[df_mid['setting'] == 'Drug', 'setting'] = None
df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,Gross Charge,De-Identified Minimum,De-Identified Maximum,payer_name,standard_charge,hospital_id
1,inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,"80,887.44",NaN,NaN,NaN,NaN,393303
2,inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,"70,558.50",NaN,NaN,NaN,NaN,393303
3,inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,"127,150.82",NaN,NaN,NaN,NaN,393303
4,inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,"438,620.29",NaN,NaN,NaN,NaN,393303
5,inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,"277,411.38",NaN,NaN,NaN,NaN,393303
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,NaN,NaN,NaN,Preferred Health Care,"278,286.91",393300
269965,inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"9,429.04",393300
269966,inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"31,395.05",393300
269967,inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"151,876.07",393300


In [10]:
df_mid['code'] = df_mid['code'].fillna('')

def extract_apr_drg(code):
    apr_drg = code.split(' ')[-1]
    apr_drg = apr_drg[:3] + '-' + apr_drg[-1]
    return apr_drg

df_mid.loc[df_mid['code'].str.startswith('PA/APR DRG'), 'apr_drg'] = df_mid[df_mid['code'].str.startswith('PA/APR DRG')]['code'].apply(extract_apr_drg)
df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,Gross Charge,De-Identified Minimum,De-Identified Maximum,payer_name,standard_charge,hospital_id,apr_drg
1,inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,"80,887.44",NaN,NaN,NaN,NaN,393303,027-1
2,inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,"70,558.50",NaN,NaN,NaN,NaN,393303,027-2
3,inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,"127,150.82",NaN,NaN,NaN,NaN,393303,027-3
4,inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,"438,620.29",NaN,NaN,NaN,NaN,393303,027-4
5,inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,"277,411.38",NaN,NaN,NaN,NaN,393303,029-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,NaN,NaN,NaN,Preferred Health Care,"278,286.91",393300,860-4
269965,inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"9,429.04",393300,862-1
269966,inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"31,395.05",393300,862-2
269967,inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"151,876.07",393300,862-3


In [11]:
df_mid.loc[df_mid['code'].str.startswith('CPT'), 'hcpcs_cpt'] = df_mid[df_mid['code'].str.startswith('CPT')]['code'].apply(lambda code: code.split(' ')[-1])
df_mid.loc[df_mid['code'].str.startswith('HCPCS'), 'hcpcs_cpt'] = df_mid[df_mid['code'].str.startswith('HCPCS')]['code'].apply(lambda code: code.split(' ')[-1].upper())
df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].fillna('')
df_mid.loc[df_mid['hcpcs_cpt'].str.len() == 6, 'hcpcs_cpt'] = None
df_mid.loc[df_mid['hcpcs_cpt'] == '', 'hcpcs_cpt'] = None
df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,Gross Charge,De-Identified Minimum,De-Identified Maximum,payer_name,standard_charge,hospital_id,apr_drg,hcpcs_cpt
1,inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,"80,887.44",NaN,NaN,NaN,NaN,393303,027-1,None
2,inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,"70,558.50",NaN,NaN,NaN,NaN,393303,027-2,None
3,inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,"127,150.82",NaN,NaN,NaN,NaN,393303,027-3,None
4,inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,"438,620.29",NaN,NaN,NaN,NaN,393303,027-4,None
5,inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,"277,411.38",NaN,NaN,NaN,NaN,393303,029-1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,NaN,NaN,NaN,Preferred Health Care,"278,286.91",393300,860-4,None
269965,inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"9,429.04",393300,862-1,None
269966,inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"31,395.05",393300,862-2,None
269967,inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,NaN,NaN,NaN,Preferred Health Care,"151,876.07",393300,862-3,None


In [12]:
df_mid['payer_category'] = 'payer'

df_mid.loc[df_mid[' Gross Charge '].notnull(), 'standard_charge'] = df_mid[df_mid[' Gross Charge '].notnull()][' Gross Charge ']
df_mid.loc[df_mid[' Gross Charge '].notnull(), 'payer_category'] = 'gross'
df_mid.loc[df_mid[' Gross Charge '].notnull(), 'payer_name'] = 'Gross Charge'
del df_mid[' Gross Charge ']

df_mid.loc[df_mid[' De-Identified Minimum '].notnull(), 'standard_charge'] = df_mid[df_mid[' De-Identified Minimum '].notnull()][' De-Identified Minimum ']
df_mid.loc[df_mid[' De-Identified Minimum '].notnull(), 'payer_category'] = 'min'
df_mid.loc[df_mid[' De-Identified Minimum '].notnull(), 'payer_name'] = 'De-Identified Minimum'
del df_mid[' De-Identified Minimum ']

df_mid.loc[df_mid[' De-Identified Maximum '].notnull(), 'standard_charge'] = df_mid[df_mid[' De-Identified Maximum '].notnull()][' De-Identified Maximum ']
df_mid.loc[df_mid[' De-Identified Maximum '].notnull(), 'payer_category'] = 'max'
df_mid.loc[df_mid[' De-Identified Maximum '].notnull(), 'payer_name'] = 'De-Identified Maximum'
del df_mid[' De-Identified Maximum ']

df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,payer_name,standard_charge,hospital_id,apr_drg,hcpcs_cpt,payer_category
1,inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,Gross Charge,"80,887.44",393303,027-1,None,gross
2,inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,Gross Charge,"70,558.50",393303,027-2,None,gross
3,inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,Gross Charge,"127,150.82",393303,027-3,None,gross
4,inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,Gross Charge,"438,620.29",393303,027-4,None,gross
5,inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,Gross Charge,"277,411.38",393303,029-1,None,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,Preferred Health Care,"278,286.91",393300,860-4,None,payer
269965,inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,Preferred Health Care,"9,429.04",393300,862-1,None,payer
269966,inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,Preferred Health Care,"31,395.05",393300,862-2,None,payer
269967,inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,Preferred Health Care,"151,876.07",393300,862-3,None,payer


In [13]:
df_mid['standard_charge'] = df_mid['standard_charge'].fillna('')
df_mid['standard_charge'] = df_mid['standard_charge'].astype(str)

df_mid.loc[df_mid['standard_charge'].str.contains('Transplant'), 'additional_payer_specific_notes'] = df_mid[df_mid['standard_charge'].str.contains('Transplant')]['standard_charge'].str.strip()
df_mid.loc[df_mid['standard_charge'].str.contains('Invoice'), 'additional_payer_specific_notes'] = df_mid[df_mid['standard_charge'].str.contains('Invoice')]['standard_charge'].str.strip()
df_mid.loc[df_mid['standard_charge'].str.contains('Transplant'), 'standard_charge'] = ''
df_mid.loc[df_mid['standard_charge'].str.contains('Invoice'), 'standard_charge'] = ''

df_mid['standard_charge'] = df_mid['standard_charge'].apply(cleanup_dollar_value)

df_mid.loc[df_mid['standard_charge'] == '', 'standard_charge'] = None
df_mid.loc[df_mid['standard_charge'] == 'nan', 'standard_charge'] = None

df_mid = pd.DataFrame(df_mid[df_mid['standard_charge'] != '-'])

df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,payer_name,standard_charge,hospital_id,apr_drg,hcpcs_cpt,payer_category,additional_payer_specific_notes
1,inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,Gross Charge,80887.44,393303,027-1,None,gross,NaN
2,inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,Gross Charge,70558.50,393303,027-2,None,gross,NaN
3,inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,Gross Charge,127150.82,393303,027-3,None,gross,NaN
4,inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,Gross Charge,438620.29,393303,027-4,None,gross,NaN
5,inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,Gross Charge,277411.38,393303,029-1,None,gross,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,Preferred Health Care,278286.91,393300,860-4,None,payer,NaN
269965,inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,Preferred Health Care,9429.04,393300,862-1,None,payer,NaN
269966,inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,Preferred Health Care,31395.05,393300,862-2,None,payer,NaN
269967,inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,Preferred Health Care,151876.07,393300,862-3,None,payer,NaN


In [14]:
df_mid['rev_code'] = None
df_mid['ms_drg'] = None
df_mid['eapg'] = None
df_mid['modifiers'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['billing_class'] = None
df_mid['plan_name'] = None
df_mid['standard_charge_percent'] = None
df_mid['contracting_method'] = None

df_mid

,setting,line_type,additional_generic_notes,local_code,code,description,ndc,payer_name,standard_charge,hospital_id,...,apc,icd,drug_hcpcs_multiplier,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,plan_name,standard_charge_percent,contracting_method
1,inpatient,DRG,NaN,PAAPR0271,PA/APR DRG V37 0271,Other Open Craniotomy,NaN,Gross Charge,80887.44,393303,...,None,None,None,None,None,None,None,None,None,None
2,inpatient,DRG,NaN,PAAPR0272,PA/APR DRG V37 0272,Other Open Craniotomy,NaN,Gross Charge,70558.50,393303,...,None,None,None,None,None,None,None,None,None,None
3,inpatient,DRG,NaN,PAAPR0273,PA/APR DRG V37 0273,Other Open Craniotomy,NaN,Gross Charge,127150.82,393303,...,None,None,None,None,None,None,None,None,None,None
4,inpatient,DRG,NaN,PAAPR0274,PA/APR DRG V37 0274,Other Open Craniotomy,NaN,Gross Charge,438620.29,393303,...,None,None,None,None,None,None,None,None,None,None
5,inpatient,DRG,NaN,PAAPR0291,PA/APR DRG V37 0291,Other Percutaneous Intracranial Procedures,NaN,Gross Charge,277411.38,393303,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,inpatient,DRG,NaN,PAAPR8604,PA/APR DRG V37 8604,Rehabilitation,NaN,Preferred Health Care,278286.91,393300,...,None,None,None,None,None,None,None,None,None,None
269965,inpatient,DRG,NaN,PAAPR8621,PA/APR DRG V37 8621,Other Aftercare & Convalescence,NaN,Preferred Health Care,9429.04,393300,...,None,None,None,None,None,None,None,None,None,None
269966,inpatient,DRG,NaN,PAAPR8622,PA/APR DRG V37 8622,Other Aftercare & Convalescence,NaN,Preferred Health Care,31395.05,393300,...,None,None,None,None,None,None,None,None,None,None
269967,inpatient,DRG,NaN,PAAPR8623,PA/APR DRG V37 8623,Other Aftercare & Convalescence,NaN,Preferred Health Care,151876.07,393300,...,None,None,None,None,None,None,None,None,None,None


In [15]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
1,393303,DRG,Other Open Craniotomy,None,PAAPR0271,PA/APR DRG V37 0271,None,027-1,None,None,...,None,inpatient,gross,Gross Charge,None,80887.44,None,None,NaN,NaN
2,393303,DRG,Other Open Craniotomy,None,PAAPR0272,PA/APR DRG V37 0272,None,027-2,None,None,...,None,inpatient,gross,Gross Charge,None,70558.50,None,None,NaN,NaN
3,393303,DRG,Other Open Craniotomy,None,PAAPR0273,PA/APR DRG V37 0273,None,027-3,None,None,...,None,inpatient,gross,Gross Charge,None,127150.82,None,None,NaN,NaN
4,393303,DRG,Other Open Craniotomy,None,PAAPR0274,PA/APR DRG V37 0274,None,027-4,None,None,...,None,inpatient,gross,Gross Charge,None,438620.29,None,None,NaN,NaN
5,393303,DRG,Other Percutaneous Intracranial Procedures,None,PAAPR0291,PA/APR DRG V37 0291,None,029-1,None,None,...,None,inpatient,gross,Gross Charge,None,277411.38,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269964,393300,DRG,Rehabilitation,None,PAAPR8604,PA/APR DRG V37 8604,None,860-4,None,None,...,None,inpatient,payer,Preferred Health Care,None,278286.91,None,None,NaN,NaN
269965,393300,DRG,Other Aftercare & Convalescence,None,PAAPR8621,PA/APR DRG V37 8621,None,862-1,None,None,...,None,inpatient,payer,Preferred Health Care,None,9429.04,None,None,NaN,NaN
269966,393300,DRG,Other Aftercare & Convalescence,None,PAAPR8622,PA/APR DRG V37 8622,None,862-2,None,None,...,None,inpatient,payer,Preferred Health Care,None,31395.05,None,None,NaN,NaN
269967,393300,DRG,Other Aftercare & Convalescence,None,PAAPR8623,PA/APR DRG V37 8623,None,862-3,None,None,...,None,inpatient,payer,Preferred Health Care,None,151876.07,None,None,NaN,NaN


In [16]:
df_out.to_csv('rate_chop.csv', index=False)